# Here is a model for NLP that allows us to clean data and correctly categorize each post from reddit as it was classified.  Step by step the model will:
1) clean and process data
2) vectorize data
3) fit a model with spacy
4) score the model with a confusion matrix
5) test the model
6) pickle the model

In [140]:
# Imports
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import pandas as pd
import string
import re
import nltk

In [141]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tcnick12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
# Load in data, mined by Jonathan
url = 'https://raw.githubusercontent.com/BW-Post-Here-01/DS/master/Data/reddit_data_slimmed.csv'
df = pd.read_csv(url)

In [143]:
# Visualize df
df.head()

,content,subreddit
0,COMMUNITY ANNOUNCEMENT In solidarity with the ...,tattoos
1,Weekly r/Tattoos Question/FreeTalk Thread! - A...,tattoos
2,Enter Shikari and Architects album artwork and...,tattoos
3,"David Bowie Portrait - Healed, Done in April 2...",tattoos
4,Photo realism artist chicago As the title sugg...,tattoos


In [144]:
# Process data with this function
def cleaning_fn(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    4. Returns in lowercase.
    """
    # Check characters to see if they are in punctuation
    clean = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    clean = ''.join(clean)
    
    clean = clean.lower()
    
    # Now just remove any stopwords
    return [word for word in clean.split() if word.lower() not in stopwords.words('english')]

In [145]:
# Randomize the rows of the df so we don't have the iloc 1-100 all classified
# as one class, the next 200 as another, etc., so we don't have issues with
# a train/test split
df = df.sample(frac=1).reset_index(drop=True)

In [146]:
# Show the df['subreddit'] is no longer grouped by class but it sorted at random.
df.head()

,content,subreddit
0,"ELI5: When washing my hands, why does really h...",explainlikeimfive
1,"I panicked when I lost my son at Disney World,...",nosleep
2,TIFU by permanently burning myself trying to c...,tifu
3,Last dance documentary for Korea So the last d...,nba
4,I Was Forced to be a Mother to a Doll Despite ...,nosleep


# The actual predictive model with three examples:

In [148]:
# Apply train/test split
X = df['content']
y = df['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(24048,)
(8017,)
(24048,)
(8017,)


In [149]:
# Create pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer=cleaning_fn)),
    ('classifier', RandomForestClassifier()),  # Originally trained with MulinomialNB() but had low accuracy
])

In [150]:
# Fit X_train and y_train on the pipe
pipeline.fit(X_train,y_train)

C:\Users\tcnick12\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer=<function cleaning_fn at 0x000002AE0593D4C8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                             

In [151]:
# Create a pipeline prediction object with X_test
predictions = pipeline.predict(X_test)

# Score the model with X_test and y_test
print(classification_report(y_test,predictions))

C:\Users\tcnick12\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                   precision    recall  f1-score   support

          Android       0.31      0.28      0.29        76
              DIY       0.07      0.04      0.05        50
          Fitness       0.53      0.63      0.57       300
       Futurology       0.16      0.08      0.11        37
            Games       0.19      0.10      0.13        41
     GetMotivated       0.40      0.24      0.30        17
             IAmA       0.64      0.92      0.76       313
            Jokes       0.24      0.46      0.31       462
      LifeProTips       0.69      0.83      0.75       203
  MachineLearning       0.66      0.70      0.68       198
            Music       0.38      0.14      0.21        97
        Overwatch       0.37      0.21      0.27       112
              PS4       0.41      0.49      0.45       179
   Showerthoughts       0.25      0.02      0.04        42
           Tinder       0.50      0.16      0.24        25
  TwoXChromosomes       0.28      0.23      0.25       

In [152]:
# Looking at these categories, try four fake reviews and see how the model does:
df.subreddit.unique()

array(['explainlikeimfive', 'nosleep', 'tifu', 'nba', 'personalfinance',
       'television', 'leagueoflegends', 'Jokes', 'Fitness', 'buildapc',
       'TwoXChromosomes', 'relationships', 'gaming', 'pokemon',
       'dadjokes', 'askscience', 'webdev', 'travel', 'atheism', 'history',
       'lifehacks', 'Overwatch', 'LifeProTips', 'PS4', 'MachineLearning',
       'IAmA', 'Music', 'gadgets', 'malefashionadvice', 'space', 'books',
       'pcmasterrace', 'Futurology', 'DIY', 'Games', 'GetMotivated',
       'trees', 'movies', 'gameofthrones', 'WritingPrompts', 'Android',
       'soccer', 'listentothis', 'Tinder', 'philosophy', 'tattoos',
       'Showerthoughts', 'technology', 'politics', 'europe'], dtype=object)

In [153]:
# Create fn that takes in a reddit post and returns the top five most likely categories:
def get_predictions(post, num_answers=5):
  """ takes a post and returns the top categories it fits in """

  # get the predicted probabilities for each class
  preds = pd.Series(pipeline.predict_proba(post)[0])

  # save each class to the Series index
  preds.index = pipeline.classes_

  # sort to get the most likely classes
  preds = preds.sort_values(ascending=False)

  # return the top num_answers results in dict format
  return preds[:5]

# Test one with a fake review about history

In [154]:
# Test with a fake review
history_post = [ """
                History if my favorite subject.  I love to read historical accounts about ancient Rome and Greece.
                I'm also a big World War 2 buff and I collect objects with historical significance.
                """]

In [155]:
get_predictions(history_post)

history              0.5
PS4                  0.2
Fitness              0.1
nba                  0.1
explainlikeimfive    0.1
dtype: float64

# Test two with a fake review about pokemon

In [156]:
# Try again and mention pokemon to see if the model correctly guesses pokemon:
pokemon_post = [ """
                My favorite pokemon are pikachu and charizard.
                """]

In [157]:
get_predictions(pokemon_post)

pokemon     0.4
dadjokes    0.3
Jokes       0.3
webdev      0.0
PS4         0.0
dtype: float64

# Test three with a fake post about android

In [158]:
# Try a fake prediction to see if android gets predicted:
android_post = [ """
                I use a galaxy note 5.  My favorite opperating system version was oreo.
                Android phones are better than iphones. I like to create apps for the app store.
                """]

In [159]:
get_predictions(android_post)

lifehacks      0.3
Android        0.2
nba            0.1
Jokes          0.1
LifeProTips    0.1
dtype: float64

# Test four with a fake post about music

In [160]:
# Try a fake prediction to see if music gets predicted:
music_post = [ """
                I love to listen to music.  My favorite singer/songwriter is Foy Vance.  Every so often
                I like to listen to Bob Marley.  I have a large vinyl music collection but more recently I've
                been listening to everything on Spotify.
                """]

In [161]:
get_predictions(music_post)

Music      0.3
webdev     0.1
Games      0.1
history    0.1
nba        0.1
dtype: float64

# Pickle the model:

In [162]:
from pickle import dump
# save the model
dump(pipeline, open('reddit_model.pkl', 'wb'))

# How to load in the model again:

In [138]:
from pickle import load
# load the model
loaded_model = load(open('reddit_model.pkl', 'rb'))

# For the Flask app API

In [ ]:
# Example code meant to be in a Flask app.  Won't run on colab

from pickle import load
# load the model
loaded_model = load(open('reddit_model.pkl', 'rb'))


from flask import jsonify

@app.route("/predict.json", methods=["POST"])
def predict():
  print("PREDICT ROUTE...")
  print("FORM DATA:", dict(request.form))
  #> {'title': 'example title', 'text': 'Example reddit post text here'}

  # concatenate title and text, passed in as one variable to the model
  post = request.form["title"] + ' ' + screen_name_b = request.form["text"]

  # get predictions, store as a Pandas Series
  preds = pd.Series(loaded_model.predict_proba(music_post)[0])

  # assign the subreddit classes to the index
  preds.index = loaded_model.classes_

  # sort by values to get the top results
  preds = preds.sort_values(ascending=False)

  # return the top 5 results as JSON
  return jsonify(subreddits=preds.index[:5],
                  probabilities=preds[:5])

# sklearn Version:

In [139]:
import sklearn
sklearn.__version__

'0.21.3'